In [ ]:
pip install mediapipe opencv-python pyautogui


In [ ]:
pip install pygame


In [ ]:
pip install sounddevice numpy


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
from tkinter import messagebox
import sounddevice as sd
import threading  # To handle Tkinter in a different thread

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize warning counter
warning_count = 0
max_warnings = 100  # For every 100th warning, play a beep

# Function to generate and play a beep sound
def play_beep(frequency=500, duration=5, volume=0.2):
    sample_rate = 44100  # Sample rate for audio (44.1 kHz)
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    wave = 0.5 * np.sin(2 * np.pi * frequency * t)  # Generate sine wave
    sd.play(wave * volume, sample_rate)  # Play sound
    sd.wait()  # Wait until sound is done playing

# Function to show warning pop-up
def show_warning(warning_count):
    def display_warning():
        root = tk.Tk()
        root.withdraw()  # Hide the main Tkinter window
        message = f"This is your {warning_count} warning. Press OK to continue."
        messagebox.showinfo("Posture Warning", message)
        root.destroy()  # Close the window after OK is pressed
    
    # Run the warning in a separate thread to avoid blocking the main loop
    threading.Thread(target=display_warning).start()

# Function to check shoulder alignment and spine straightness
def check_posture(landmarks):
    # Extract relevant landmarks (left and right shoulders, hips)
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    # Detect slouch or forward shoulder (side view detection) by checking horizontal difference between shoulder and hip
    spine_slouch_threshold = 0.1  # Threshold for slouching (forward shoulder posture)
    shoulder_hip_diff = abs(left_shoulder.x - left_hip.x)  # Check horizontal alignment between shoulder and hip

    spine_straight = shoulder_hip_diff < spine_slouch_threshold  # If the difference is large, the user is slouching
    print(f"Spine alignment (side view) detected: {spine_straight} (Shoulder-Hip Difference: {shoulder_hip_diff})")

    # If the user is leaning too far forward (indicating slouching), it's bad posture
    return not spine_straight

# Main function
def main():
    global warning_count

    cap = cv2.VideoCapture(0)  # Open webcam
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            landmarks = results.pose_landmarks.landmark

            # Check posture based on shoulder alignment and spine straightness
            bad_posture_detected = check_posture(landmarks)

            if bad_posture_detected:
                warning_count += 1
                print(f"Posture warning {warning_count}/{max_warnings}")

                # Show a warning pop-up for the first nine warnings
                if warning_count < max_warnings:
                    show_warning(warning_count)

                # Play a beep sound after every 10th warning
                if warning_count >= max_warnings:
                    play_beep()  # Beep for 5 seconds
                    warning_count = 0  # Reset the warning count after the beep
            else:
                warning_count = 0

        # Display posture warning text if warnings exist
        if warning_count > 0:
            cv2.putText(frame, f"Posture Warning {warning_count}/{max_warnings}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show the video feed
        cv2.imshow('Posture Correction', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
